## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import folium
import geopandas as gpd
import glob
from datetime import datetime, timedelta

## Data Read-in

In [2]:
df = pd.read_csv('2023 eviction notices.xlsx - Sheet1.csv')

In [3]:
df['File Date'] = pd.to_datetime(df['File Date'])

In [4]:
df_filtered = df[df['File Date'] > '5.31.23']

In [5]:
df_filtered = df_filtered.drop_duplicates(subset='Eviction ID')

In [6]:
columns_to_check = ['Non Payment', 'Breach',
       'Nuisance', 'Illegal Use', 'Failure to Sign Renewal', 'Access Denial',
       'Unapproved Subtenant', 'Owner Move In', 'Demolition',
       'Capital Improvement', 'Substantial Rehab', 'Ellis Act WithDrawal',
       'Condo Conversion', 'Roommate Same Unit', 'Other Cause',
       'Late Payments', 'Lead Remediation', 'Development',
       'Good Samaritan Ends']

In [7]:
df_filtered['Eviction_Reason'] = df_filtered[columns_to_check].idxmax(axis=1)

In [8]:
df_filtered['Eviction_Reason'].value_counts()

Eviction_Reason
Nuisance                158
Non Payment              43
Capital Improvement      36
Breach                   35
Owner Move In            30
Ellis Act WithDrawal     24
Other Cause              11
Roommate Same Unit        8
Unapproved Subtenant      7
Illegal Use               1
Access Denial             1
Late Payments             1
Name: count, dtype: int64

## HTML Popup Formatter

In [9]:
def popup_html(row):
    Eviction_ID = row['Eviction ID']
    Address = row['Address']
    Neighborhood = row['Neighborhoods - Analysis Boundaries']
    Eviction_Reason = row['Eviction_Reason']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Eviction ID: </strong>{}'''.format(Eviction_ID) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>Neighborhood: </strong>{}'''.format(Neighborhood) + '''<br>
    <strong>Eviction Reason: </strong>{}'''.format(Eviction_Reason) + '''<br>
    </html>
    '''
    return html

In [10]:
df_filtered.columns

Index(['Eviction ID', 'Address', 'City', 'State',
       'Eviction Notice Source Zipcode', 'File Date', 'Non Payment', 'Breach',
       'Nuisance', 'Illegal Use', 'Failure to Sign Renewal', 'Access Denial',
       'Unapproved Subtenant', 'Owner Move In', 'Demolition',
       'Capital Improvement', 'Substantial Rehab', 'Ellis Act WithDrawal',
       'Condo Conversion', 'Roommate Same Unit', 'Other Cause',
       'Late Payments', 'Lead Remediation', 'Development',
       'Good Samaritan Ends', 'Constraints Date', 'Supervisor District',
       'Neighborhoods - Analysis Boundaries', 'Location', 'Shape',
       'SF Find Neighborhoods', 'Current Police Districts',
       'Current Supervisor Districts', 'Analysis Neighborhoods',
       'DELETE - Neighborhoods', 'DELETE - Police Districts',
       'DELETE - Supervisor Districts', 'DELETE - Fire Prevention Districts',
       'DELETE - Zip Codes', 'CBD, BID and GBD Boundaries as of 2017',
       'Central Market/Tenderloin Boundary', 'Areas of Vu

In [11]:
# Remove parentheses and split the 'Location' column
df_filtered[['LATITUDE', 'LONGITUDE']] = df_filtered['Location'].str.replace('[()]', '', regex=True).str.split(', ', expand=True)

# Convert 'LATITUDE' and 'LONGITUDE' to numeric types
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'], errors='coerce')
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'], errors='coerce')

In [12]:
len(df_filtered)

355

In [13]:
df_filtered = df_filtered.dropna(subset=['LATITUDE','LONGITUDE'])

In [14]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=11,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"San Francisco eviction notices for the second half of 2023")

m.get_root().html.add_child(folium.Element(title_html))

for index, row in df_filtered.iterrows():
    marker = folium.Marker(
        location=[row['LATITUDE'], row['LONGITUDE']],
        radius=5,
        fill=True,
        popup=folium.Popup(popup_html(row), max_width=400))
    marker.add_to(m)

# Add the FeatureGroups to the map

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map
m

In [15]:
m.save('index.html')

## Map URL Snagger

In [16]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/SF_2023_eviction_notices_July_onwards
